# Excel Files And How To Make Toy Datasets

Before we get started. Somebody recently asked me about Excel files and how to make smaller datasets.  To load an Excel file, you may need to add an extra dependency.  (Technially, the module for loading Excel files is an optional dependency in Pandas.  If you do not have it, you will get a lot of errors when you try to load an xlsx file.)  So let's add that dependency:

    conda install openpyxl

And, then you can import Excel files into Paandas as:

In [16]:
import pandas as pd

And then let's load a data set with missing values.  It may take a minute or two to load:

In [10]:
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx')

Let's take a quick look at this dataset.

In [12]:
df.describe()

,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


Okay, so this kind of a big data set and could be a bit hard to use as an example, so you can sample a smaller set from the data.   Let's grab 100 elements of the data, sample without replacement (and specify the random seed so that we all have the same dataset).  Note that you can also use the `frac` argument instead of `n` if you want a fraction of the data and don't care about how many.

In [22]:
df_sample = online_retail_df.sample(n=100, replace=False, random_state=1)
df_sample.describe()

,Quantity,Price,Customer ID
count,100.000000,100.000000,83.000000
mean,5.870000,2.772200,15425.710843
std,17.160946,2.751018,1680.258143
min,-127.000000,0.000000,12471.000000
25%,1.000000,1.150000,13813.000000
50%,3.000000,2.025000,15353.000000
75%,12.000000,3.377500,16835.000000
max,60.000000,15.950000,18069.000000


# Missing Values

Now on to the main topic!  So far, the data that we receive as input is never missing any values, but, that is not always the case.  Today, we will look at what we can do if we are missing values.  So, let's make a data set with some missing values!  So, let's start off and add some data with missing values.

In [24]:
import pandas as pd
import numpy as np

In [26]:
df = pd.DataFrame([
    [0, 1, 0, 0, 0, 1],
    [2, 0, 1, 1, 0, 0],
    [None, 0, 2, 0, 1, 0],
    [3, 1, None, 0, 1, 0],
    [None, 0, 2, 1,0,0]]
, columns = ['Atr1', 'Atr2','Atr3', 'Atr4_A', 'Atr4_B', 'Atr4_C'])
df

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0
2,NaN,0,2.0,0,1,0
3,3.0,1,NaN,0,1,0
4,NaN,0,2.0,1,0,0


Note that any value that was a `None` is a `NaN` in the data frame.  That means that we have a missing value.  We can count the number of missing values per attribtue as:

In [28]:
df.isnull().sum()

Atr1      2
Atr2      0
Atr3      1
Atr4_A    0
Atr4_B    0
Atr4_C    0
dtype: int64

The easiest way to not deal with missing data is to simply get rid of it!  So, let's drop them:

In [34]:
df.dropna(axis='rows')

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0


So that resulted in us dropping all the rows that have missing data.  Alternatively, we could drop all the columns with missing data.  While we already know how to take a subset of columns in a data frame, this is very common and so we can specify an argument to `DataFrame.dropna` to do it for us:

In [35]:
df.dropna(axis='columns')

,Atr2,Atr4_A,Atr4_B,Atr4_C
0,1,0,0,1
1,0,1,0,0
2,0,0,1,0
3,1,0,1,0
4,0,1,0,0


Alterntively, maybe we don't want to get rid of the data, but try to fill in values.  So for example, set all NaN's to zero with:

In [36]:
df.fillna(0)

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0
2,0.0,0,2.0,0,1,0
3,3.0,1,0.0,0,1,0
4,0.0,0,2.0,1,0,0


Other fill techniques include fill forward (fill using the value in the previous row) or backward fill (fill using the value in the next row)

In [40]:
df.fillna(method='ffill')

   Atr1  Atr2  Atr3  Atr4_A  Atr4_B  Atr4_C
0   0.0     1   0.0       0       0       1
1   2.0     0   1.0       1       0       0
2   2.0     0   2.0       0       1       0
3   3.0     1   2.0       0       1       0
4   3.0     0   2.0       1       0       0


In [41]:
df.fillna(method='bfill')

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0
2,3.0,0,2.0,0,1,0
3,3.0,1,2.0,0,1,0
4,NaN,0,2.0,1,0,0


Note that the last line didn't get filled becuase there is nothing to fill it with!  (As always, be careful, boundaries are always tricky!)

So, let'd get creative, a really natural way to fill is to use the mean of the data that we have.  Let's try that.  First, let's get the mean of the first column, save it into a varible, and set.

In [59]:
mean_x0 = np.mean(df.values[:,0])
df['Atr1'].fillna(mean_x0)

0    0.0
1    2.0
2    NaN
3    3.0
4    NaN
Name: Atr1, dtype: float64

Well that was not as exciting as we were hoping, what happened.... Lets look at the value in the mean:

In [56]:
np.mean(df.values[:,0])

nan

So it seems that numpy didn't ignore the missing values and once we get one NaN, they cascade.... so Let's compute the mean without the NaN's

In [64]:
mean_x0 = np.nanmean(df.values[:, 0])
mean_x0

1.6666666666666667

Which is what we expect if we sum up column 0 with all values that we have and divide by the number of values we have... (0 + 2 + 3) / 3

So now, lets fill all columns with their means.  See if you can get it in a few lines.

In [125]:
mm = np.nanmean(df.values, axis=0)
df.fillna(value={'Atr1': mm[0], 'Atr3': mm[2]})

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.000000,1,0.00,0,0,1
1,2.000000,0,1.00,1,0,0
2,1.666667,0,2.00,0,1,0
3,3.000000,1,1.25,0,1,0
4,1.666667,0,2.00,1,0,0


Some of the above was for illustrative purposes.  Really the fastest way (to my knowledge) 

In [124]:
df.fillna(value=df.mean())

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.000000,1,0.00,0,0,1
1,2.000000,0,1.00,1,0,0
2,1.666667,0,2.00,0,1,0
3,3.000000,1,1.25,0,1,0
4,1.666667,0,2.00,1,0,0


But, what if a value is categorical (or perhaps numeric but integral or boolean). Then, mean makes little sense.  Let's use mode!

In [146]:
df.fillna(value=df.mode())

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0
2,3.0,0,2.0,0,1,0
3,3.0,1,NaN,0,1,0
4,NaN,0,2.0,1,0,0


Well, that ws not what we expected... let's explore.

In [147]:
df.mode()

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,0.0,2.0,0.0,0.0,0.0
1,2.0,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN


Well that is somewhat unexpected as well... why all the rows? it is becuase mode is the element with the most number of entires.  We can have multiple values for the mode.  In the first column we have 0, 2, 3 each one time.  So, let's just grab the first row

In [150]:
df.mode().iloc[0]

Atr1      0.0
Atr2      0.0
Atr3      2.0
Atr4_A    0.0
Atr4_B    0.0
Atr4_C    0.0
Name: 0, dtype: float64

and put it all together as

In [151]:
df.fillna(value=df.mode().iloc[0])

,Atr1,Atr2,Atr3,Atr4_A,Atr4_B,Atr4_C
0,0.0,1,0.0,0,0,1
1,2.0,0,1.0,1,0,0
2,0.0,0,2.0,0,1,0
3,3.0,1,2.0,0,1,0
4,0.0,0,2.0,1,0,0


And we can do lots of other types methods as well.  A nice introduction to some additional methods can be found at https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779

**Next up** Graphs! (for real this time)